In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_427161/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/12 14:53:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/12 14:53:02 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for the Student table
student_data = {
    'student_id': [1, 2, 3],
    'student_name': ['Jack', 'Jane', 'Mark'],
    'gender': ['M', 'F', 'M'],
    'dept_id': [1, 1, 2]
}

# Sample data for the Department table
department_data = {
    'dept_id': [1, 2, 3],
    'dept_name': ['Engineering', 'Science', 'Law']
}

# Creating pandas DataFrames for Student and Department tables
student_df = pd.DataFrame(student_data)
department_df = pd.DataFrame(department_data)

# Displaying the Student and Department DataFrames
print("Student table:")
print(student_df)
print("\nDepartment table:")
print(department_df)



df_person = spark.createDataFrame(student_df)
df_person.createOrReplaceTempView("Student")

df_person = spark.createDataFrame(department_df)
df_person.createOrReplaceTempView("Department")



Student table:
   student_id student_name gender  dept_id
0           1         Jack      M        1
1           2         Jane      F        1
2           3         Mark      M        2

Department table:
   dept_id    dept_name
0        1  Engineering
1        2      Science
2        3          Law


In [ ]:
query = """
    select
        d.dept_name,
        ifnull(s.student_number, 0) as student_number
    from Department d
    left join (
        select
            dept_id,
            count(*) as student_number
        from Student s
        group by dept_id
    ) s on d.dept_id = s.dept_id

"""

In [4]:
query = """
select dept_name,count(Student.student_id)
from Department
left join Student on Department.dept_id = Student.dept_id
group by dept_name

"""

In [5]:
spark.sql(query).show()

+-----------+-----------------+
|  dept_name|count(student_id)|
+-----------+-----------------+
|    Science|                1|
|Engineering|                2|
|        Law|                0|
+-----------+-----------------+

